Complex Scheduling, Summer term 2023

*Prof. Dr. Rainer Kolisch, M.Sc. Pia Ammann*

# Assignment 1: PSP

**[15 points]**

In this assignment, you should implement a simple optimization model for a PSP and algorithms for computing earliest and latest start and finish times. Data structures and methods for data input and output are given.

**Instructions**:
Please submit your solution together with the names and matriculation numbers of all team members until Sunday, 05-14-2023 (23:59) via email to pia.ammann@tum.de.
Use "*CXS23 Assignment 1 [last name student 1 / last name student 2 (/last name student 3 )]*" for the subject line.
In addition, please provide a "*statement of contribution*", listing each team member's individual contribution to the submission. 

## Basics 

This section introduces the basic data structures and shows how to read the instance file(s). Please read the descriptions carefully and execute all code cells to make use of the defined variables.

### Data Structures

An object of the class `Network` represents a network of activities with the (dummy) starting node $0$, the (dummy) finishing node $n+1$ and $n$ activity nodes. All precedence relations and timelags are stored in the attribute arcs (dictionary).

An object of class `Node` represents a node with an index and attributes earliest start time (est), latest start time (lst), earliest finish time (eft), latest finish time (eft), total float (tf), early free float (eff), late free float (lff), and start time (start). Furthermore, every node has a set of successors and a processing time (fixed to 0 for this assignemnt).

In [1]:
# data structures
class Node:
    
    def __init__(self, index, successors):
        self.id=index               #identifier for the node
        self.successors=successors  #successors: a list of node ids follow this node
        self.processing_time=0      #processing time of the node, initialize as 0
        self.est=None     # earliest / latest start / finish times
        self.lst=None
        self.eft=None     # earliest finishing time
        self.lft=None     # latest finishing time
        self.tf=None      # total floats
        self.eff=None     # early free floats
        self.lff=None     # latest free float
        self.start=None   # start time of the node
        
    def __str__(self):
        return 'A node with index {} and successors {}'.format(self.id, 
                                                               self.successors)
    
    def __repr__(self):
        return self.id, self.successors

    
class Network:
    
    def __init__(self, name, node_ids, arcs, Tmax):
        self.name = name                # name of the network
        self.node_ids = node_ids        # list of node ids representing the node in the network
                                        # dictionary map the node ids to Node objects
        self.node_dict = {n: Node(index=n, successors=[i for i in node_ids if (n,i) in arcs]) for n in self.node_ids}
        self.arcs=arcs                  # dict of timelags between arcs in the network
        self.Tmax=Tmax                  # maximum time for the network
        self.makespan=None              # makespan of the network
        # additional attributes for RCPSP
        self.resourceavailability = []  #a list contain availability of resource
        self.number_of_resources = None #the number of resources
        
    def __str__(self):
        return '{}: A network with {}+2 nodes, {} arcs, and deadline {}.'.format(self.name, len(self.node_ids)-2, len(self.arcs), self.Tmax)
    
    def __repr__(self):
        return self.name, self.node_ids, self.arcs, self.Tmax
    
        

### Read Data

We now import our test data from the Excel file PSP_data.xlsx located in the folder Assignment_1. The provided Microsoft Excel document contains three test instances of projects with 10, 100 and 1000 activities. Those are derived from instances of the [RCPSP/max instance library](https://www.wiwi.tu-clausthal.de/abteilungen/betriebswirtschaftslehre-insbesondere-produktion-und-logistik/forschung/schwerpunkte/single-mode-project-duration-problem-rcpsp/max).

All project instances are stored in a dictionary named `Instances`, sheet names are used as keys.

In [2]:
# mount drive to access data files
#from google.colab import drive
#drive.mount('/content/drive')
#error happens so import locally instead
!pip install --upgrade openpyxl
import pandas as pd
from openpyxl import load_workbook
workbook = load_workbook(filename="D:\Van\Complex Scheduling\PSP_data.xlsx")
Instances = {}

# iterate through all sheets (instances)
for sheet in workbook.worksheets:

    # get general info first
    sheet_name = sheet.title
    n = int(sheet.cell(1, 2).value)
    Tmax = int(sheet.cell(1, 4).value)

    # read time lags
    df = pd.read_excel("D:\Van\Complex Scheduling\PSP_data.xlsx", sheet_name=sheet_name, header=2,
                       usecols='A:C')

    # create dict
    arcs = list(zip(df.i, df.j))
    time_lags = dict(zip(arcs, df.delta_ij))

    Instances[sheet_name] = Network(name=sheet_name,
                                    node_ids=list(range(0,n+2)),
                                    arcs=time_lags, Tmax=Tmax)


## 1 LP Modelling Time-Constrained Project Scheduling Problem

Given is a project plan as directed graph $G=(V,E)$ with activities $V$, time lag relations $E$, arc weights $\delta_{ij} \; \forall \; (i,j)\in E$. Starting times $S_i \in \mathbb{R}$ are to be defined $\forall i \in V$. 

\begin{aligned}
&\text{Minimize } f(\mathbf{S})&\\
&s.t.\\
&S_j-S_i\geq \delta_{ij}&\forall (i,j)\in E\\
&S_i\geq 0&\forall i\in V\\
&S_0=0&\\
&S_{n+1}\leq T^{\text{max}}&\\
\end{aligned}



Install Gurobi and create environment with your WLS license

(https://support.gurobi.com/hc/en-us/articles/4409582394769-Google-Colab-Installation-and-Licensing)

In [3]:
# 0 install / import required packages
!pip install gurobipy  
import gurobipy as gp  
from gurobipy import GRB

In [4]:
# 1 Read license from file

lic_path = 'D:\Van\Complex Scheduling\gurobi.lic'

lic = {}
lic = 2372825

# 2 Create environment with your license
e = gp.Env(empty=True)
e.setParam('LicenseID', lic)
e.start()


Set parameter Username
Set parameter LicenseID to value 2372825
Academic license - for non-commercial use only - expires 2024-04-30


<gurobipy.Env, Parameter changes: LicenseID=2372825>

a.) Implement the method `solve()` which *formulates* and *solves* an optimization model of the Generic time Scheduling Problem with the objective to minimize the sum of (unweighted) starting time (WST objective function) for a given network. Use the solver Gurobi.

This method should *not* print the default solver output but instead print the name of the *test record*, the optimal *objective value* and the *runtime* of the solver in seconds.

Furthermore, this method should set the nodes attributes (`start`) and network attributes (`makespan`) accordingly.

**[2.5 points]**

In [5]:
def solve(network):
    #Create the model within the Gurobi environment defined previously
    m = gp.Model('PSP', env=e)
    # add your variables
    S = m.addVars(network.node_ids, lb=0, ub= network.Tmax , vtype=GRB.CONTINUOUS, name='S')
    m.setObjective(S[network.node_ids[-1]], GRB.MINIMIZE)
    # add your constraints
    #for i,j in network.node_ids:
    for (i,j), v in network.arcs.items():
        m.addConstr((S[j]-S[i]) >= v)
    m.addConstr(S[network.node_ids[-1]] <= network.Tmax)
    m.addConstr(S[network.node_ids[0]] == 0)
    # define your objective
    m.optimize()
    if m.status == GRB.OPTIMAL :
        print("Optimal solution found")
        for i in network.node_ids :
            print(f"S_{i} = {S[i].x}")
    else:
        print("Optimal solution not found")


In [1]:
def compute_time_windwos(network):

    # Step 1: Initialize all est values to 0
    for node_id in network.node_ids:
        network.node_dict[node_id].est = 0

    # Step 2: Perform Label-Correcting Algorithm
    queue = [network.node_dict[1]]  # Assuming start node has ID 1
    visited = set()

    while queue: # While queue is not empty
        current_node = queue.pop(0) # Pop first element from queue
        visited.add(current_node) # Add current node to visited set

        for successor_id in current_node.successors: # For all successors of current node
            successor = network.node_dict[successor_id] # Get successor node object
            if successor.est < current_node.est + successor.d # If successor.est < current_node.est + successor.processing_time
                successor.est = current_node.est + successor.processing_time # Set successor.est to current_node.est + successor.processing_time

            if successor not in visited: # If successor not in visited
                queue.append(successor) # Append successor to queue

    # Step 3: Set lst values to the latest possible start time (equal to est)
    for node_id in network.node_ids: # For all nodes
        network.node_dict[node_id].lst = network.node_dict[node_id].est # Set lst to est
        print("Node:", node_id)
        print("est:", network.node_dict[node_id].est)
        print("lst:", network.node_dict[node_id].lst)
        print()

SyntaxError: invalid syntax (2466418954.py, line 17)

b.) Write a loop, which solves the three test records stored in `Instances` one after another, making use of the method `solve()` defined previously. 

**[1 point]**

In [11]:
for sheet_name in Instances:
    network = Instances[sheet_name]
    compute_time_windwos(network)

Node: 0
est: 0
lst: 0

Node: 1
est: 0
lst: 0

Node: 2
est: 0
lst: 0

Node: 3
est: 0
lst: 0

Node: 4
est: 0
lst: 0

Node: 5
est: 0
lst: 0

Node: 6
est: 0
lst: 0

Node: 7
est: 0
lst: 0

Node: 8
est: 0
lst: 0

Node: 9
est: 0
lst: 0

Node: 10
est: 0
lst: 0

Node: 11
est: 0
lst: 0

Node: 0
est: 0
lst: 0

Node: 1
est: 0
lst: 0

Node: 2
est: 0
lst: 0

Node: 3
est: 0
lst: 0

Node: 4
est: 0
lst: 0

Node: 5
est: 0
lst: 0

Node: 6
est: 0
lst: 0

Node: 7
est: 0
lst: 0

Node: 8
est: 0
lst: 0

Node: 9
est: 0
lst: 0

Node: 10
est: 0
lst: 0

Node: 11
est: 0
lst: 0

Node: 12
est: 0
lst: 0

Node: 13
est: 0
lst: 0

Node: 14
est: 0
lst: 0

Node: 15
est: 0
lst: 0

Node: 16
est: 0
lst: 0

Node: 17
est: 0
lst: 0

Node: 18
est: 0
lst: 0

Node: 19
est: 0
lst: 0

Node: 20
est: 0
lst: 0

Node: 21
est: 0
lst: 0

Node: 22
est: 0
lst: 0

Node: 23
est: 0
lst: 0

Node: 24
est: 0
lst: 0

Node: 25
est: 0
lst: 0

Node: 26
est: 0
lst: 0

Node: 27
est: 0
lst: 0

Node: 28
est: 0
lst: 0

Node: 29
est: 0
lst: 0

Node: 30
est

In [6]:
# insert your code for part 1 b.) here:
for sheet_name in Instances:
    network = Instances[sheet_name]
    solve(network)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25 rows, 12 columns and 48 nonzeros
Model fingerprint: 0x4faa2406
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+01, 2e+01]
  RHS range        [2e+00, 2e+01]
Presolve removed 25 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.800000000e+01
Optimal solution found
S_0 = 0.0
S_1 = 0.0
S_2 = 0.0
S_3 = 0.0
S_4 = 5.0
S_5 = 9.0
S_6 = 4.0
S_7 = 0.0
S_8 = 0.0
S_9 = 3.0
S_10 = 2.0
S_11 = 18.0
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (w

## 2 Calculating earliest and latest start and finish times

### 2.1 Label Correcting Algorithm

Compute earliest and latest start and finish times with the Label-Correcting-Algorithm (LCA) as introduced in the lecture (see pp.25 - 32).

a.) Implement the method `lca()` that calculates the **longest paths** between a single node and all other nodes in a given network $N = (V, E, \delta_{ij})$  (as introduced on p. 25 of the lecture slides). 

This method takes a network / project as input and returns two lists $d_{0,i}$ and $p_{0,i} \; \forall \; i \in V$.

**[2 points]**

In [12]:
def lca(network):
  
  # init
  d = [0] + [float('-inf') for n in network.node_ids[1:]] # list for distances
  p = [0] + [-1 for n in network.node_ids[1:]]            # list for predecessors
  q = [0]                                                 # queue

  # insert your code for part 2.1 a.) here
  #
  while q:
      i = q.pop(0)  # dequeue
      for j in network.node_dict[i].successors:
          dist = d[i] + network.arcs[(i, j)]
          if dist > d[j]:
              d[j] = dist
              p[j] = i
              if d[i] > (len(network.node_ids) -1) * max(network.arcs.values()):
                  return("infeasible")
              if j not in q:
                  q.append(j)  # enqueue
  return d, p, q
for sheet_name in Instances:
    network = Instances[sheet_name]
    print(sheet_name + "------------------------------------------------")
    print(lca(network))



psp10_1------------------------------------------------


KeyError: (0, 1)

b.) Implement the method `create_auxiliary_network()` that returns an **auxiliary network** ($N^H$) as described on p. 30 in the lecture slides (Steps 1 to 2). 

**[1.5 points]**

In [8]:
import copy

In [9]:
# transform network for computing LS, LF
def create_auxiliary_network(network):
  #step 1 Step 1: Adding arc (𝑛 + 1,0) with 𝛿𝑛+1,0 = −𝑇𝑚𝑎𝑥
    network.node_dict[network.node_ids[-1]].successors.append(network.node_ids[0])
    network.arcs[(network.node_ids[-1], network.node_ids[0])] = -network.Tmax
  # Step 2: Reverse the arcs
    network.arcs = {(j, i): v for (i, j), v in network.arcs.items()}
    return network
for sheet_name in Instances:
    net_temp = copy.deepcopy(Instances[sheet_name])
    print(create_auxiliary_network(net_temp).arcs)

{(3, 0): 0, (2, 0): 0, (1, 0): 0, (8, 0): 0, (10, 1): 2, (4, 2): 5, (11, 2): 9, (7, 2): 0, (9, 3): 3, (11, 4): 6, (5, 4): 4, (11, 5): 9, (6, 5): -5, (5, 6): -4, (7, 6): -4, (11, 6): 10, (8, 7): -4, (11, 7): 5, (11, 8): 7, (7, 8): -2, (11, 9): 7, (11, 10): 5, (1, 10): -3, (0, 11): -20}
{(4, 0): 0, (7, 0): 0, (8, 0): 0, (2, 0): 0, (6, 0): 0, (5, 0): 0, (3, 0): 0, (1, 0): 0, (9, 0): 0, (29, 0): 0, (78, 1): 2, (16, 1): 10, (29, 2): -2, (80, 2): 24, (37, 2): 27, (69, 3): 14, (91, 4): 20, (67, 4): 12, (47, 5): 23, (39, 5): -7, (94, 5): 11, (76, 6): 17, (53, 6): 6, (85, 6): 16, (90, 7): 7, (12, 8): 0, (89, 8): 5, (82, 8): 16, (21, 9): 25, (99, 9): 7, (11, 9): 3, (95, 10): 13, (21, 11): 10, (53, 11): 6, (74, 11): 8, (99, 12): 6, (58, 12): 5, (74, 12): 6, (52, 13): -25, (50, 13): 8, (44, 13): -25, (65, 14): 13, (92, 14): -1, (30, 14): -150, (18, 14): -131, (20, 15): 13, (27, 15): -255, (73, 15): -2, (55, 16): 13, (94, 16): 0, (68, 17): 3, (85, 17): 10, (40, 18): -82, (92, 18): 3, (86, 18): 20, 

c.) Define a method `compute_elst_lca()` that computes **earliest and latest start and end times** for all nodes in the given network using the LCA. This method should make use of the two previously defined methods `transform_network()` and `lca()`. It takes a network / project $N$ as input and sets the node attributes (est, eft, lst, lft) accordingly. 

**[1.5 points]**

In [10]:
# ES, EF, LS, LF
def compute_elst_lca(network):
    # insert your code for part 2.1 c.) here
  # Step 1: Transform the network
    network = create_auxiliary_network(network)
  # Step 2: Compute the longest paths
    d, p, q = lca(network)
  # Step 3: Compute the ES, EF, LS, LF
    es = [d[i] for i in network.node_ids]
    ef = [d[i] + network.arcs[(i, j)] for i, j in network.arcs.keys()]
    # Step 4: Compute latest start and finish times
    lf = [d[-1] - d[i] for i in network.node_ids]
    ls = [d[j] - network.arcs[(i, j)] for i, j in network.arcs.keys()]
    return es, ef, ls, lf
network = Instances["psp10_1"]
compute_elst_lca(network)

KeyError: (0, 1)

d.) Finally, run the code cell below to test your implementation on all instances and to write your results into a separate sheet. (might take a few minutes!)

In [ ]:
# Apply to test instances
sol = {}
for k, i in Instances.items():
  
  # compute ES, EF, LS, LF with the LCA
  compute_elst_lca(i)

  # store your solutions
  est, eft, lst, lft, s = [], [], [], [], []
  for n in i.node_dict.values():
    est.append(n.est)
    eft.append(n.eft)
    lst.append(n.lst)
    lft.append(n.lft)
    s.append(n.start)
  sol[i.name] = pd.DataFrame
  sol[i.name] = pd.DataFrame(data={'EST':est, 'EFT':eft, 'LST':lst, 'LFT':lft, 'S':s},
                              index=i.node_dict.keys())

# write solution dataframe to file (e.g., xlsx)
sol_file = '/content/drive/My Drive/CXS23_ipynb/1_Assignment/PSP_elst.xlsx'
writer = pd.ExcelWriter(sol_file) # mode='a' to keep existing sheets
for i, df in sol.items():
  df.to_excel(writer, sheet_name="LCA_{}".format(i))
writer.save()


### 2.2 Floyd-Warshall Algorithm
Compute earliest and latest start and finish times with the Floyd-Warshall Algorithm (Triple Algorithm) as introduced in the lecture (see pp. 33 - 42).

a.) Define the method `fwa()` that calculates the longest paths between all nodes in a given network $N = (V, E, \delta_{ij})$ (lecture slides p. 33). 

This method takes a network / project $N$ as input and returns two dictionaries $d_{i,j}$ and $p_{i,j}$ for all $i, j \in V$.

**[3 points]**

In [ ]:
def fwa(network):
  
  # init
  d = {i: {j: float("inf") for j in network.node_ids} for i in network.node_ids}
  p = {i: {j: None for j in network.node_ids} for i in network.node_ids}

  for v in network.node_ids:
      for i in network.node_ids:
          for j in network.node_ids:
              if d[i][v] + d[v][j] < d[i][j]:
                  d[i][j] = d[i][v] + d[v][j]
                  p[i][j] = p[v][j]

      # Check for negative cycles
      if d[v][v] < 0:
          raise ValueError("Negative cycle detected")
      
  return d, p
print(fwa(network))

b.) Define the method `compute_elst_fwa()` that computes **earliest and latest start and end times** for all nodes in the given network using the FWA. This method should make use of the previously defined method `fwa()`. It takes a network / project $N$ as input and sets the node attributes (est, eft, lst, lft) accordingly.

**[1 point]**

In [ ]:
# ES, EF, LS, LF
def compute_elst_fwa(network):

        d, p = fwa(network)
        #print(d)
        for k, v in network.node_dict.items():
            v.est = d[k]
            v.eft = d[k] + p[k]
        auxiliary_network = create_auxiliary_network(network)
        s, t = fwa(auxiliary_network)
        #print(s)
        #print(t)
        for k, v in network.node_dict.items():
            #print(d)
            v.lst = -s[k][k]
            v.lft = v.lst + v.duration
            #print(network.node_dict[k].lst)
            #print(network.node_dict[k].lft)
        return



c.) Finally, run the code cell below to test your implementation on all instances and to write your results into a separate sheet. (might take a few minutes!)

In [ ]:
#import openpyxl

# Apply to test instances
sol = {}
for k, i in Instances.items():
  
  # compute ES, EF, LS, LF with the LCA
  compute_elst_fwa(i)

  # store your solutions
  est, eft, lst, lft, s = [], [], [], [], []
  for n in i.node_dict.values():
    est.append(n.est)
    eft.append(n.eft)
    lst.append(n.lst)
    lft.append(n.lft)
    s.append(n.start)
  sol[i.name] = pd.DataFrame
  sol[i.name] = pd.DataFrame(data={'EST':est, 'EFT':eft, 'LST':lst, 'LFT':lft, 'S':s},
                              index=i.node_dict.keys())

# write solution dataframe to file (e.g., xlsx)
for i, df in sol.items():
  df.to_excel(writer, sheet_name="FWA_{}".format(i))
writer.save()


## 3 Calculating float times

Calculate the float times, which define the amount of time by which the (earliest/latest) start of an activity can be delayed without delaying succeeding activities / the latest completion of the project. See *worksheet 1* for details.

a.) Implement the method `compute_floats()` that computes the total float, the early free float, and the late free float for all nodes in a given network. This method takes a network / project $N$ as input and sets the node attributes (tf, eff, lff) accordingly.

**[2.5 point]**

In [ ]:
# TF, EFF, LFF
def compute_floats(network):

    compute_elst_fwa(network)

    for k, v in network.node_dict.items():
        v.tf = v.lst - v.est
        #print(network.node_dict[2].tf)
        min = network.Tmax
        for (k, j) in network.arcs.keys():
            if (network.node_dict[j].est - network.arcs[(k, j)] - network.node_dict[k].est) < min :
                min = network.node_dict[j].est - network.arcs[(k, j)] - network.node_dict[k].est
        network.node_dict[k].eff = min

        max = 0
        for (j, k) in network.arcs.keys():
            if (network.node_dict[j].lst + network.arcs[(j, k)]) > max:
                max = network.node_dict[j].lst + network.arcs[(j, k)]
                network.node_dict[k].lff = network.node_dict[k].lst - max

            #print(network.node_dict[2])
            #print(network.node_dict[2].eff)
    return

b.) Finally, run the code cell below to test your implementation on all instances and to write your results into a separate sheet. (might take a few minutes!)

In [2]:
#import openpyxl

# Apply to test instances
sol = {}
for k, i in Instances.items():
  
  # compute ES, EF, LS, LF with the LCA
  compute_floats(i)

  # store your solutions
  tf, eff, lff = [], [], []
  for n in i.node_dict.values():
    tf.append(n.tf)
    eff.append(n.eff)
    lff.append(n.lff)
  sol[i.name] = pd.DataFrame
  sol[i.name] = pd.DataFrame(data={'TF':est, 'EFF':eft, 'LFF':lst},
                              index=i.node_dict.keys())

# write solution dataframe to file (e.g., xlsx)
for i, df in sol.items():
  df.to_excel(writer, sheet_name="FLOATS_{}".format(i))
writer.save()

NameError: name 'Instances' is not defined

Close your session (run code cell below)

In [3]:
# unmount google drive
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

NameError: name 'drive' is not defined